In [2]:
import pandas as pd
from tqdm import tqdm
from neo4j import GraphDatabase
import json
import re
import os
import glob

In [4]:

path = "./"
all_files = glob.glob(os.path.join(path, "./Datasets/api_comic_list*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged   = pd.concat(df_from_each_file, ignore_index=True)
df_merged.to_csv( "./Datasets/merged.csv")

In [6]:
db_host = "bolt://localhost:7687"
password = "1234qwer"


In [7]:
api_characters = pd.read_csv("./Datasets/api_character_list.csv")
scraping_characters = pd.read_csv(
    "./Datasets/scraping_personaggi_completo_filtered_relations.csv")
api_comics = pd.read_csv("./Datasets/merged.csv")


In [ ]:
scraping_characters.head(20)

In [ ]:
api_characters.head(20)

Add characters nodes with variant from API

In [8]:
def add_character_node(tx, name, character_id, character_description):
  processed_name = name.replace('"', " ").replace('"'," ")
  nome_senza_parentesi = re.sub("[\(\[].*?[\)\]]", "", processed_name.strip())
  query_string = 'MERGE (c:character{name: "%s"})' %(nome_senza_parentesi.replace("'", " ").lower().strip())
  tx.run(query_string)
  tx.run(
      "MERGE (n:character_variant {name: '%s', character_id: %s, character_description: '%s'})" %(name.replace("'", ' ').replace('"', " ").replace("-", " "), character_id, str(character_description).replace("'", ' ').replace('"', " ").replace("-", " ")))
  query_variant = 'MATCH (a:character{name:"%s"}),(b:character_variant {name: "%s", character_id: %s, character_description: "%s"}) MERGE (b)-[r:variante_di]->(a)' % (
      nome_senza_parentesi.replace("'", " ").lower().strip(), name.replace("'", ' ').replace('"', " ").replace("-", " ").strip(), character_id, str(character_description).replace("'", ' ').replace('"', " ").replace("-", " ").strip(),)
  tx.run(query_variant)


In [9]:
def add_comic_node_and_link(tx, comic_title, comic_id, character_id, comic_description):
  comic_description = str(comic_description).replace("'", " ").replace('"',"").replace("-"," ").strip()
  create_comic_node_query = 'MERGE (c:comic {title:"%s", comic_id: %s, comic_description: "%s"})' % (
      comic_title, comic_id, comic_description)
  create_link_query = "MATCH (a:character_variant),(b:comic) WHERE a.character_id = %s AND b.comic_id = %s MERGE (a)-[r:Presente]->(b)" % (
      character_id, comic_id)
  # print(create_link_query)
  node_creation_result = tx.run(create_comic_node_query)
  link_creation_result = tx.run(create_link_query)


In [10]:
def add_comic_node(tx, comic_title,comic_id, comic_description):
  comic_description = str(comic_description).replace(
      "'", " ").replace('"', "").replace("-", " ").strip()
  create_comic_node_query = 'MERGE (c:comic {title:"%s", comic_id: %s, comic_description: "%s"})' % (
      comic_title, comic_id, comic_description)
  node_creation_result = tx.run(create_comic_node_query)


In [11]:
driver = GraphDatabase.driver(db_host, auth=("neo4j",password))

In [65]:

for idx, row in tqdm(api_characters.iterrows(), total=api_characters.shape[0]):
    with driver.session() as session:
        try:
            session.write_transaction(
                add_character_node, row['name'], row["id"], row["description"])
        except Exception as e:
            print(e
            )

100%|██████████| 1559/1559 [01:02<00:00, 24.76it/s]


In [12]:
for idx, row in tqdm(api_comics.iterrows(), total=api_comics.shape[0]):
    string = row["characters"]
    
    characters = json.loads(string)
    if characters["items"]:
        for character in characters["items"]:
            with driver.session() as session:
                try:
                    character_id = character["resourceURI"].split("/")
                    character_id = character_id[len(character_id) -1]
                    session.write_transaction(
                            add_comic_node_and_link, row['title'], row["id"], character_id,  row["description"])
                except Exception as e:
                    print(e)
    else:
        with driver.session() as session:
            try:
                session.write_transaction(
                    add_comic_node, row['title'], row["id"],  row["description"])
            except Exception as e:
                print(e)
    


 47%|████▋     | 24048/50645 [35:56<1:05:28,  6.77it/s] 

{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'N': expected whitespace, '.', node labels or rel types, '[', '^', '*', '/', '%', '+', '-', "=~", IN, STARTS, ENDS, CONTAINS, IS, '=', "<>", "!=", '<', '>', "<=", ">=", AND, XOR, OR, ',' or '}' (line 1, column 40 (offset: 39))
"MERGE (c:comic {title:"Stephen King's "N." (2010) #4", comic_id: 31322, comic_description: "As the mysterious case of N. spreads and claims the lives of everyone who hears the story each victim tries helplessly to rescue the next victim from their own sealed fate. But when the odd tale of Ackerman s field finds its way into the hands of a journalist one with millions of readers there is no telling how far the madness of N. might spread. DON T MISS A MOMENT OF THIS GRIPPING SERIES FROM MASTER THRILLER STEPHEN KING, WITH SCRIPT BY MARC GUGGENHEIM AND ART BY ALEX MALEEV!"
                                        ^}


 48%|████▊     | 24258/50645 [36:56<52:28,  8.38it/s]   

{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'N': expected whitespace, '.', node labels or rel types, '[', '^', '*', '/', '%', '+', '-', "=~", IN, STARTS, ENDS, CONTAINS, IS, '=', "<>", "!=", '<', '>', "<=", ">=", AND, XOR, OR, ',' or '}' (line 1, column 40 (offset: 39))
"MERGE (c:comic {title:"Stephen King's "N." (2010) #3", comic_id: 31321, comic_description: "STEPHEN KING PRESENTS A TERRIFYING TALE OF MAGIC AND MADNESS! Can you catch a mental illness the same way you can catch a cold? Until he met patient N., Doctor John Bonsaint would have said that such an idea was ridiculous. But following the suicide of his patient, Bonsaint is finding the curious case of N. is plaguing him night and day. He begins exhibiting the same symptoms as N. insomnia, obsessive compulsive tendencies and finds the questions of N.s all consuming condition. But how far down the rabbit hole will he follow his patient...and what monsters lie at the bottom? [Brought to life by MARC GUG

 48%|████▊     | 24474/50645 [37:14<41:58, 10.39it/s]  

{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'N': expected whitespace, '.', node labels or rel types, '[', '^', '*', '/', '%', '+', '-', "=~", IN, STARTS, ENDS, CONTAINS, IS, '=', "<>", "!=", '<', '>', "<=", ">=", AND, XOR, OR, ',' or '}' (line 1, column 40 (offset: 39))
"MERGE (c:comic {title:"Stephen King's "N." (2010) #2", comic_id: 31319, comic_description: "Patient Case Study: N. Subject initially complained of insomnia, stating that standard pharma was not having any effect. I will prescribe him something stronger. Subject is also open and frank about obsessive compulsive tendencies, most notably; he has a habit of counting, and speaks of good and bad numbers. Traces symptoms back to visit of certain field in rural Maine with a rock formation that changes in number. Calls it a faded place where he could tear through the fabric of reality. Also recalls a grinning figure trapped within the stones. Suggests strong bouts of delusion. Prognosis: stronger sedat

 49%|████▉     | 24690/50645 [37:33<40:22, 10.71it/s]  

{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'N': expected whitespace, '.', node labels or rel types, '[', '^', '*', '/', '%', '+', '-', "=~", IN, STARTS, ENDS, CONTAINS, IS, '=', "<>", "!=", '<', '>', "<=", ">=", AND, XOR, OR, ',' or '}' (line 1, column 40 (offset: 39))
"MERGE (c:comic {title:"Stephen King's "N." (2010) #1", comic_id: 31316, comic_description: "There is something unearthly and mysterious deep in Ackerman s Field in rural Maine. There is a Stonehenge like arrangement of seven stones with a horrifying EYE in the center. And whatever dwells there in that strange, windswept setting may have brought about the suicide of one man...and harbor death for the OCD afflicted N., whose visits to the field have passed beyond compulsion into the realm of obsession. Based on the chilling short story from the recent Stephen King collection, JUST AFTER SUNSET, this adaptation will provide nightmares aplenty. Just keep counting the stones...keep counting...count

100%|██████████| 50645/50645 [1:49:06<00:00,  7.74it/s]   
